Pull a selection of MPC observations and up to three orbits and save the data locally.

To access MPC data you will need a Gcloud project with subscription to 2 BigQuery sources:
Main MPC Dataset (Analytics Hub listing) and Clustered Views Dataset (Analytics Hub listing).
Here we assume that subscription ids were the defaults, and the project id is provided as
an enviroment variable.

After this notebook is run, `data/orbit_fit_eval` will contain 4 Parquet files with
MPC observations, MPC orbits, SBDB orbits, and NEOCC orbits.

In [ ]:
from typing import List

import os

from adam_core.orbits.query import query_neocc, query_sbdb
from mpcq.client import BigQueryMPCClient
from pathlib import Path

In [ ]:
# The default path is relative to this notebook's location
def fetch_and_save(object_ids: List, out_dir: Path = Path("../data/orbit_fit_eval")):
    out_dir.mkdir(parents=True, exist_ok=True)

    client = BigQueryMPCClient(
        dataset_id="asteroid_institute_mpc_replica",
        views_dataset_id="asteroid_institute___mpc_replica_views",
        project=os.environ["MPCQ_PROJECT_ID"],
    )

    observations = client.query_observations(object_ids)
    observations.to_parquet(out_dir / "mpc_observations.parquet")

    mpc_orbits = client.query_orbits(object_ids)
    mpc_orbits.to_parquet(out_dir / "mpc_orbits.parquet")

    sbdb_orbits = query_sbdb(object_ids)
    sbdb_orbits.to_parquet(out_dir / "sbdb_orbits.parquet")

    neocc_orbits = query_neocc(object_ids, orbit_type="ke", orbit_epoch="present-day")
    neocc_orbits.to_parquet(out_dir / "neocc_orbits.parquet")

In [ ]:
desired_object_ids = [
    "1992 LR",   # Akka, AMO, no NEOCC orbit, small number of nasty outliers in observations
    "2026 BN6",  # APO, new one, small number of observations, almost all with uncertainty, short arc length, has NEOCC
    "2025 WZ9",  # ATE, new one, has NEOCC
    "2013 UU15", # MCA, 6 stations, but 3 with <5 observations
]
fetch_and_save(desired_object_ids)